In [1]:
import jupyterlab_dash
import dash
import dash_html_components as html

import psycopg2
import pandas as pd
from plotly.offline import init_notebook_mode, iplot
from plotly.offline import plot 
import plotly.graph_objects as go
import warnings
import datetime
warnings.filterwarnings('ignore')

import dash
import dash_table
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
from dash.exceptions import PreventUpdate
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# import flask

# import q_credentials.db_secmaster_cloud_cred as db_secmaster_cred
import q_credentials.db_secmaster_cred as db_secmaster_cred
import q_credentials.db_indicator_cred as db_indicator_cred

In [2]:
# connect to our securities_master database
conn_secmaster = psycopg2.connect(host=db_secmaster_cred.dbHost , database=db_secmaster_cred.dbName, user=db_secmaster_cred.dbUser, password=db_secmaster_cred.dbPWD)
conn_indicator = psycopg2.connect(host=db_indicator_cred.dbHost , database=db_indicator_cred.dbName, user=db_indicator_cred.dbUser, password=db_indicator_cred.dbPWD)

sql="""SELECT s.ticker, max(d.symbol_id), max(d.date_price)
FROM public.symbol s inner join d_data d on s.id= d.symbol_id
group by s.ticker"""
df_ticker_last_day=pd.read_sql(sql,con=conn_secmaster)

def level_plot(df):
    support_ls = [[ls[0],ls[1],datetime.datetime.strptime(ls[2],'%Y-%m-%d %H:%M:%S'),ls[3]] for ls in df.iloc[-1]['level_support']]
    resistance_ls = [[ls[0],ls[1],datetime.datetime.strptime(ls[2],'%Y-%m-%d %H:%M:%S'),ls[3]] for ls in df.iloc[-1]['level_resistance']]
    end_dt=df.index[-1]
    res_plot_ls=[]
    sup_plot_ls=[]
    for res in resistance_ls[:5]:
        res_plot_ls.append(dict(x0=res[2],x1=end_dt,y0=res[0],y1=res[1],yref='y1',opacity=.2,fillcolor='Red',line=dict(color="black",width=1)))
    for sup in support_ls[:5]:
        sup_plot_ls.append(dict(x0=sup[2],x1=end_dt,y0=sup[0],y1=sup[1],yref='y1',opacity=.2,fillcolor='green',line=dict(color="black",width=1)))
    return (res_plot_ls+sup_plot_ls)

def data_selector(symbol_id):
    sql="select * from indicator"
    ind_list=list(pd.read_sql(sql,con=conn_indicator)['name'])
#     symbol_id='EUR_USD'#'AAPL'#'BOM500114'
    start_date=datetime.datetime(2018,1,1).strftime("%Y-%m-%d")
#     indicator_name = 'candle_1'
    
    sql="select d.date_price as date, open_price as open, high_price as high, low_price as low , close_price as close,volume from d_data d join symbol s on d.symbol_id = s.id where s.ticker='%s' and d.date_price > '%s'" %(symbol_id, start_date)
    df_price=pd.read_sql(sql,con=conn_secmaster)
    
    df_all_ind=pd.DataFrame()
    for ind in ind_list:
        print(ind)
        sql="select d.date_price as date, d.value from d_data d join symbol s on d.symbol_id = s.id join indicator i on i.id=d.indicator_id where s.ticker='%s' and i.name = '%s' and d.date_price > '%s'" %(symbol_id, ind, start_date)
        df_indicator=pd.read_sql(sql,con=conn_indicator)
        df_indicator.set_index('date',inplace=True)
        df_indicator=pd.concat([df_indicator.drop(['value'], axis=1), df_indicator['value'].apply(pd.Series)], axis=1)
        df_indicator.columns=[ind+"_"+col for col in df_indicator.columns]
        if df_all_ind.empty:
            df_all_ind=pd.merge(left=df_price, right=df_indicator,on='date')
        else:
            df_all_ind=pd.merge(left=df_all_ind, right=df_indicator,on='date')
    
    df_all_ind.set_index('date',inplace=True)
    return df_all_ind

In [3]:
df_vendor = pd.read_csv('Data/vendor.csv')
df_vendor.dropna(inplace=True)
df_vendor.head()

,Unnamed: 0,Unnamed: 0.1,Col 1,Col 2,Col 3,Col 4,Col 5,Col 6,Col 7,Col 8,...,Col 21,Col 22,Col 23,Col 24,Col 25,Col 26,Col 27,Col 28,Col 29,Col 30
0,0,0,0.340773,a,b,a,b,a,b,0.034522,...,0.460611,0.876137,0.890906,0.785446,0.157581,0.967033,0.780561,0.479723,0.358613,0.073457
1,1,1,0.774953,b,c,b,c,b,c,0.846041,...,0.746636,0.962178,0.201873,0.815196,0.346221,0.110245,0.125527,0.510152,0.418297,0.163809
2,2,2,0.495336,a,b,a,b,a,b,0.768301,...,0.925783,0.625773,0.008629,0.503926,0.906269,0.465226,0.662788,0.461829,0.648599,0.194076
3,3,3,0.381686,b,c,b,c,b,c,0.086562,...,0.869467,0.602467,0.640168,0.276679,0.864281,0.490874,0.037928,0.871404,0.807910,0.454460
4,4,4,0.702263,a,b,a,b,a,b,0.694671,...,0.963614,0.151442,0.903086,0.821413,0.657959,0.276269,0.116238,0.582393,0.068209,0.288750


In [4]:
df_dataset = pd.read_csv('Data/dataset.csv')
df_dataset.dropna(inplace=True)
df_dataset.head()

,Unnamed: 0,Unnamed: 0.1,Col 1,Col 2,Col 3,Col 4,Col 5,Col 6,Col 7,Col 8,...,Col 21,Col 22,Col 23,Col 24,Col 25,Col 26,Col 27,Col 28,Col 29,Col 30
0,0,0,0.340773,a,b,a,b,a,b,0.034522,...,0.460611,0.876137,0.890906,0.785446,0.157581,0.967033,0.780561,0.479723,0.358613,0.073457
1,1,1,0.774953,b,dssaasdas,b,c,b,c,0.846041,...,0.746636,0.962178,0.201873,0.815196,0.346221,0.110245,0.125527,0.510152,0.418297,0.163809
2,2,2,0.495336,a,b,a,b,a,b,0.768301,...,0.925783,0.625773,0.008629,0.503926,0.906269,0.465226,0.662788,0.461829,0.648599,0.194076
3,3,3,0.381686,b,c,b,c,b,c,0.086562,...,0.869467,0.602467,0.640168,0.276679,0.864281,0.490874,0.037928,0.871404,0.807910,0.454460
4,4,4,0.702263,a,b,a,b,a,b,0.694671,...,0.963614,0.151442,0.903086,0.821413,0.657959,0.276269,0.116238,0.582393,0.068209,0.288750


In [23]:
viewer = jupyterlab_dash.AppViewer()

external_stylesheets = ['https://codepen.io/amyoshino/pen/jzXypZ.css']
app = dash.Dash(
    __name__,
    external_stylesheets=external_stylesheets
)


# app.layout = dhc.Div([
#     dhc.Div([
#         dcc.Graph(id="plot-candle",figure=Currentfig)
#     ],style = {'display': 'inline-block', 'width': '100%','height':'200%'},className='row'),
#     dhc.Div(
#         [
#             dhc.Div([dcc.Dropdown(id='dropdown-securities',
#                     options=[{'label': i, 'value': i} for i in df_ticker_last_day['ticker'].unique()], multi=False, value="EUR_USD")],className='six columns')
# #             ,dhc.Div([dcc.Dropdown(id='dropdown-indicators',
# #                     multi=True,options=[{'label': i, 'value': i} for i in indicators])],className='six columns')
                     
#     ],className='row'),
# ])

params = [
    'Weight', 'Torque', 'Width', 'Height',
    'Efficiency', 'Power', 'Displacement'
]



app.layout = html.Div([
html.Div([
    dcc.Tabs(id="tabs-styled-with-props", value='tab-1', children=[
        dcc.Tab(label='1', value='tab-1'),
        dcc.Tab(label='2', value='tab-2'),
    ], colors={
        "border": "white",
        "primary": "gold",
        "background": "cornsilk"
    }),
    html.Div(id='tabs-content-props')
]),
# html.Div([
#     html.H1(children='Meta Data Dashboard',className='nine columns'),
#     html.Img(
#                     src="Images/jefdata.PNG",
#                     className='three columns',
#                     style={
#                         'height': '15%',
#                         'width': '15%',
#                         'float': 'right',
#                         'position': 'relative',
#                         'margin-top': 10,
#                     },
#                 ),
#                 html.Div(children='''
#                         Dash: A web application framework for Python.
#                         ''',
#                         className='nine columns'
#                 )
#     ],className='row'),
# html.Div([
#     html.Div([
#     dash_table.DataTable(
#         id='vendor_table',
#         columns=(
#             [{"name": i, "id": i} for i in df_vendor.columns]
#         ),
#         data = df_vendor.to_dict('records'),
#         editable=True,
# #         page_size=10,
# #         page_action='none',
#         style_table={'height': '400px', 'overflowY': 'auto'},
#         export_format='xlsx'
# #         export_headers='display',
# #         merge_duplicate_headers=True
#     )],className='twelve columns')
#     ],className='row'),
    
# html.Div([
#     html.Div([
#     dash_table.DataTable(
#         id='dataset_table',
#         columns=(
#             [{"name": i, "id": i} for i in df_dataset.columns]
#         ),
#         data = df_dataset.to_dict('records'),
#         editable=True,
# #         page_size=10,
# #         page_action='none',
#         style_table={'height': '400px', 'overflowY': 'auto'},
#         export_format='xlsx'
# #         export_headers='display',
# #         merge_duplicate_headers=True
#     )],className='twelve columns')
#     ],className='row'),
    
# html.Div([
#     html.Button('Save Data', id='save_file_button', n_clicks=0)
# ]),
# html.Div([
# dcc.Graph(id='table-editing-simple-output')
#     ],className='row'),

# html.Div([
# dcc.Graph(id='table-editing-simple-output2')
#     ],className='row'),
    
html.Div(id='hidden-div1', style={'display':'none'}),
html.Div(id='hidden-div2', style={'display':'none'})
])

@app.callback(Output('tabs-content-props', 'children'),
              [Input('tabs-styled-with-props', 'value')])
def render_content(tab):
    if tab == 'tab-1':
        return html.Div([
            html.H3('Tab content 1'),
            html.Div([
    html.H1(children='Meta Data Dashboard',className='nine columns'),
    html.Img(
                    src="Images/jefdata.PNG",
                    className='three columns',
                    style={
                        'height': '15%',
                        'width': '15%',
                        'float': 'right',
                        'position': 'relative',
                        'margin-top': 10,
                    },
                ),
                html.Div(children='''
                        Dash: A web application framework for Python.
                        ''',
                        className='nine columns'
                )
    ],className='row'),
html.Div([
    html.Div([
    dash_table.DataTable(
        id='vendor_table',
        columns=(
            [{"name": i, "id": i} for i in df_vendor.columns]
        ),
        data = df_vendor.to_dict('records'),
        editable=True,
#         page_size=10,
#         page_action='none',
        style_table={'height': '400px', 'overflowY': 'auto'},
        export_format='xlsx'
#         export_headers='display',
#         merge_duplicate_headers=True
    )],className='twelve columns')
    ],className='row'),
    
html.Div([
    html.Div([
    dash_table.DataTable(
        id='dataset_table',
        columns=(
            [{"name": i, "id": i} for i in df_dataset.columns]
        ),
        data = df_dataset.to_dict('records'),
        editable=True,
#         page_size=10,
#         page_action='none',
        style_table={'height': '400px', 'overflowY': 'auto'},
        export_format='xlsx'
#         export_headers='display',
#         merge_duplicate_headers=True
    )],className='twelve columns')
    ],className='row'),
    
html.Div([
    html.Button('Save Data', id='save_file_button', n_clicks=0)
]),
html.Div([
dcc.Graph(id='table-editing-simple-output')
    ],className='row'),
        ])
    elif tab == 'tab-2':
        return html.Div([
            html.H3('Tab content 2')
        ])

@app.callback(
#     Output('table-editing-simple-output', 'figure'),
    Output('hidden-div1','children'),
    [Input('save_file_button','n_clicks')],
    [State('vendor_table', 'data'),
     State('vendor_table', 'columns')])
def display_output(n_clicks,rows, columns):
    if n_clicks>0:
        df = pd.DataFrame(rows, columns=[c['name'] for c in columns])
        print("PODA")
        df.to_csv('Data/vendor.csv')
        raise PreventUpdate

@app.callback(
#     Output('table-editing-simple-output2', 'figure'),
    Output('hidden-div2','children'),
    [Input('save_file_button','n_clicks')],
    [State('dataset_table', 'data'),
     State('dataset_table', 'columns')])
def display_output_2(n_clicks, rows, columns):
    if n_clicks>0:
        df = pd.DataFrame(rows, columns=[c['name'] for c in columns])
        print("PODA")
    #     df.to_csv("Data/updated_dataset {}.csv".format(datetime.now().strftime("%m-%d-%Y-%H%M%S")))
        df.to_csv('Data/dataset.csv')
        raise PreventUpdate


In [24]:
viewer.show(app)

Dash is running on http://localhost:41681/proxy/41681/

Dash is running on http://localhost:41681/proxy/41681/

Dash is running on http://localhost:41681/proxy/41681/

Dash is running on http://localhost:41681/proxy/41681/

Dash is running on http://localhost:41681/proxy/41681/

Dash is running on http://localhost:41681/proxy/41681/

Dash is running on http://localhost:41681/proxy/41681/

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

